<a href="https://colab.research.google.com/github/madhumita-mondal/Project/blob/main/Copy_of_chatbot_ipynb_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
from google.colab import drive
drive.mount('/content/drive')
data_root = '/content/drive/My Drive/Datascience_Project/dataset/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# import json
# import string
# import random

import nltk
nltk.download('omw-1.4')
# import numpy as np
# from nltk.stem import WordNetLemmatizer
import tensorflow as tf
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")

import keras
import nltk
import pickle
import json
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
import random
import datetime
from googlesearch import *
import webbrowser
import requests
#import libarchive
#import pydot
#import cartopy
from pycricbuzz import Cricbuzz
#from pycricbuzz import Cricbuzz
# c = Cricbuzz()
import billboard
import time
from pygame import mixer
import COVID19Py

from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [36]:
!pip install COVID19Py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [38]:
!pip install billboard.py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [39]:
!pip install pycricbuzz


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [40]:
# data_file =open(data_root + 'intent_chatbot.json_old').read()
# intents = json.loads(data_file)
# print(intents)


In [41]:
# words=[]
# classes=[]
# data_x=[]
# data_y=[]
# for intent in data["intents"]:
#   for pattern in intent["patterns"]:
#     tokens = nltk.word_tokenize(pattern)
#     words.extend(tokens)
#     data_x.append(pattern)
#     data_y.append(intent["tag"])

#   if intent["tag"] not in classes:
#      classes.append(intent["tag"])

# lemmatizer= WordNetLemmatizer()

# words= [lemmatizer.lemmatize(word.lower()) for word in words if word not in string.punctuation ]
# words=sorted(set(words))
# classes=sorted(set(classes))

words=[]
classes=[]
documents=[]
ignore=['?','!',',',"'s"]

data_file =open(data_root + 'intent_chatbot.json_old').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
words=[lemmatizer.lemmatize(word.lower()) for word in words if word not in ignore]
words=sorted(list(set(words)))
classes=sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))


In [42]:
# training = []
# out_empty = [0] * len(classes)

# #Creating the bag of words model
# for idx,doc in enumerate(data_x):
#   bow=[]
#   text= lemmatizer.lemmatize(doc.lower())
#   for word in words:
#     bow.append(1) if word in text else bow.append(0)
#     output_row = list(out_empty)
#     output_row[classes.index(data_y[idx])] = 1
#     training.append([bow,output_row])
# random.shuffle(training)
# training = np.array(training,dtype=object)
# train_x=np.array(list(training[:, 0]))
# train_y=np.array(list(training[:, 1]))
#training data
training=[]
output_empty=[0]*len(classes)

for doc in documents:
    bag=[]
    pattern=doc[0]
    pattern=[ lemmatizer.lemmatize(word.lower()) for word in pattern ]
    
    for word in words:
        if word in pattern:
            bag.append(1)
        else:
            bag.append(0)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
random.shuffle(training)
training=np.array(training)  
X_train=list(training[:,0])
y_train=list(training[:,1])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [43]:
# from nltk import metrics
# from nltk.sem.evaluate import Model
# # The Neural Network Model
# model = Sequential()  
# model.add(Dense(128, input_shape = (len(train_x[0]),),activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(64, activation="relu"))
# model.add(Dropout(0.5))
# model.add(Dense(len(train_y[0]), activation="softmax"))
# adam= tf.keras.optimizers.Adam(learning_rate=0.01,decay=1e-6)
# model.compile(loss='categorical_crossentropy',
#               optimizer=adam,
#               metrics = ["accuracy"])
# #print(model.summary())
# model.fit(x=train_x, y=train_y,epochs=150,verbose=1)

#Model
model=Sequential()
model.add(Dense(128,activation='relu',input_shape=(len(X_train[0]),)))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation='softmax'))

adam = tf.keras.optimizers.Adam(0.001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
#model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)
weights=model.fit(np.array(X_train),np.array(y_train),epochs=200,batch_size=10,verbose=1)    
model.save('mymodel.h5',weights)

from keras.models import load_model
model = load_model('mymodel.h5')
data_file =open(data_root + 'intent_chatbot.json_old').read()
intents = json.loads(data_file)
#intents = json.loads(open('intent_chatbot.json_old').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))
              

Epoch 1/200
12/12 [==============================] - 1s 3ms/step - loss: 3.4326 - accuracy: 0.0254 
Epoch 2/200
12/12 [==============================] - 0s 3ms/step - loss: 3.4165 - accuracy: 0.0424
Epoch 3/200
12/12 [==============================] - 0s 3ms/step - loss: 3.3895 - accuracy: 0.1102
Epoch 4/200
12/12 [==============================] - 0s 2ms/step - loss: 3.3902 - accuracy: 0.0678
Epoch 5/200
12/12 [==============================] - 0s 3ms/step - loss: 3.3757 - accuracy: 0.0763
Epoch 6/200
12/12 [==============================] - 0s 3ms/step - loss: 3.3476 - accuracy: 0.0847
Epoch 7/200
12/12 [==============================] - 0s 2ms/step - loss: 3.3157 - accuracy: 0.1271
Epoch 8/200
12/12 [==============================] - 0s 4ms/step - loss: 3.2872 - accuracy: 0.0678
Epoch 9/200
12/12 [==============================] - 0s 6ms/step - loss: 3.2670 - accuracy: 0.1017
Epoch 10/200
12/12 [==============================] - 0s 4ms/step - loss: 3.2422 - accuracy: 0.1017
Epoch 11

In [44]:
# #Preprocessing the Input
# def clean_text(text):
#   tokens = nltk.word_tokenize(text)
#   tokens = [lemmatizer.lemmatize(word) for word in tokens]
#   return tokens
  
# def bag_of_words(text,vocab):
#   tokens = clean_text(text)
#   bow = [0] * len(vocab)
#   for w in tokens:
#     for idx, word in enumerate(vocab):
#       if word == w:
#         bow[idx]= 1
#   return np.array(bow)
# def pred_class(text,vocab,labels):
#   bow = bag_of_words(text,vocab)
#   result = model.predict(np.array([bow]))[0]
#   thresh = 0.5
#   y_pred = [[indx,res] for indx,res in enumerate(result) if res > thresh]
#   y_pred.sort(key = lambda x: x[1],reverse = True)
#   return_list=[]
#   for r in y_pred:
#     return_list.append(labels[r[0]])
#   return return_list
  
# def get_respose(intents_list,intents_json):
#   if len(intents_list) == 0:
#     result ="Sorry ! I dont understand"
#   else:
#     tag = intents_list[0]
#     list_of_intents = intents_json["intents"]
#     for i in list_of_intents:
#       if i["tag"] == tag:
#         result = random.choice(i["responses"])
#         break
#   return result

#Predict
def clean_up(sentence):
    sentence_words=nltk.word_tokenize(sentence)
    sentence_words=[ lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

def create_bow(sentence,words):
    sentence_words=clean_up(sentence)
    bag=list(np.zeros(len(words)))
    
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence,model):
    p=create_bow(sentence,words)
    res=model.predict(np.array([p]))[0]
    threshold=0.8
    results=[[i,r] for i,r in enumerate(res) if r>threshold]
    results.sort(key=lambda x: x[1],reverse=True)
    return_list=[]
    
    for result in results:
        return_list.append({'intent':classes[result[0]],'prob':str(result[1])})
    return return_list

def get_response(return_list,intents_json):
    
    if len(return_list)==0:
        tag='noanswer'
    else:    
        tag=return_list[0]['intent']
    if tag=='datetime':        
        print(time.strftime("%A"))
        print (time.strftime("%d %B %Y"))
        print (time.strftime("%H:%M:%S"))

    if tag=='google':
        query=input('Enter query...')
        chrome_path = r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe %s'
        for url in search(query, tld="co.in", num=1, stop = 1, pause = 2):
            webbrowser.open("https://google.com/search?q=%s" % query)
    if tag=='weather':
        api_key='987f44e8c16780be8c85e25a409ed07b'
        base_url = "http://api.openweathermap.org/data/2.5/weather?"
        city_name = input("Enter city name : ")
        complete_url = base_url + "appid=" + api_key + "&q=" + city_name
        response = requests.get(complete_url) 
        x=response.json()
        print('Present temp.: ',round(x['main']['temp']-273,2),'celcius ')
        print('Feels Like:: ',round(x['main']['feels_like']-273,2),'celcius ')
        print(x['weather'][0]['main'])
        
    if tag=='news':
        main_url = " http://newsapi.org/v2/top-headlines?country=in&apiKey=bc88c2e1ddd440d1be2cb0788d027ae2"
        open_news_page = requests.get(main_url).json()
        article = open_news_page["articles"]
        results = [] 
          
        for ar in article: 
            results.append([ar["title"],ar["url"]]) 
          
        for i in range(10): 
            print(i + 1, results[i][0])
            print(results[i][1],'\n')
            
    if tag=='cricket':
        c = Cricbuzz()
        matches = c.matches()
        for match in matches:
            print(match['srs'],' ',match['mnum'],' ',match['status'])
    
    if tag=='song':
        chart=billboard.ChartData('hot-100')
        print('The top 10 songs at the moment are:')
        for i in range(10):
            song=chart[i]
            print(song.title,'- ',song.artist)
    if tag=='timer':        
        mixer.init()
        x=input('Minutes to timer..')
        time.sleep(float(x)*60)
        mixer.music.load('Handbell-ringing-sound-effect.mp3')
        mixer.music.play()
        
    if tag=='covid19':
        
        covid19=COVID19Py.COVID19(data_source='jhu')
        country=input('Enter Location...')
        
        if country.lower()=='world':
            latest_world=covid19.getLatest()
            print('Confirmed:',latest_world['confirmed'],' Deaths:',latest_world['deaths'])
        
        else:
                
            latest=covid19.getLocations()
            
            latest_conf=[]
            latest_deaths=[]
            for i in range(len(latest)):
                
                if latest[i]['country'].lower()== country.lower():
                    latest_conf.append(latest[i]['latest']['confirmed'])
                    latest_deaths.append(latest[i]['latest']['deaths'])
            latest_conf=np.array(latest_conf)
            latest_deaths=np.array(latest_deaths)
            print('Confirmed: ',np.sum(latest_conf),'Deaths: ',np.sum(latest_deaths))

    list_of_intents= intents_json['intents']    
    for i in list_of_intents:
        if tag==i['tag'] :
            result= random.choice(i['responses'])
    return result

def response(text):
    return_list=predict_class(text,model)
    response=get_response(return_list,intents)
    return response

while(1):
    x=input()
    print(response(x))
    if x.lower() in ['bye','goodbye','get lost','see you']:  
        break


hello
Hi there, how can I help?
what are you doing
Talking to you, of course!
okay
Please give me more info
tell me a joke
I ate a clock yesterday, it was very time-consuming
sing a song
The top 10 songs at the moment are:
Break My Soul -  Beyonce
About Damn Time -  Lizzo
As It Was -  Harry Styles
Running Up That Hill (A Deal With God) -  Kate Bush
Wait For U -  Future Featuring Drake & Tems
First Class -  Jack Harlow
Bad Habit -  Steve Lacy
Me Porto Bonito -  Bad Bunny & Chencho Corleone
Sunroof -  Nicky Youre & dazy
Late Night Talking -  Harry Styles
...
what is your name
I am Chatter, a Deep-Learning chatbot. Your name?
mayukh
Please give me more info
my name is mayukh
You have got a such a nice name.... How can I help you?
you are dumb
Well that hurts :(
no i am sorry
Sorry, can't understand you
hiya
Please give me more info
are you tiny
Thank you!
what?
Sorry, can't understand you
are you crazy?
Thank you!
hello mr bot
Hello
werty
Not sure I understand
wertwythzosncbafccczfarrsf
N

In [45]:
# #Interacting with the chatbot
# #print("Press 0 if you want to chat with our Chatbot")
# while True:
#   message = input("")
#   if message == "0":
#     break
#   intents = pred_class (message,words,classes)
#   result = get_respose(intents,data)
#   print (result)
#Self learning
print('Help me Learn?')
tag=input('Please enter general category of your question  ')
flag=-1
for i in range(len(intents['intents'])):
    if tag.lower() in intents['intents'][i]['tag']:
        intents['intents'][i]['patterns'].append(input('Enter your message: '))
        intents['intents'][i]['responses'].append(input('Enter expected reply: '))        
        flag=1

if flag==-1:
    
    intents['intents'].append (
        {'tag':tag,
         'patterns': [input('Please enter your message')],
         'responses': [input('Enter expected reply')]})
    
with open('intents.json','w') as outfile:
    outfile.write(json.dumps(intents,indent=4))


Help me Learn?
Please enter general category of your question  no
Enter your message: thank you
Enter expected reply: thank u
Enter your message: bye
Enter expected reply: Thanks
